In [2]:
### General analysis package
import os
import time
import random
import numpy as np
import pandas as pd
import scipy.stats as stats
from scipy.stats import norm, pearsonr, spearmanr
from scipy.spatial import distance
### Drawing package
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.lines as mlines
from matplotlib.font_manager import FontProperties
#import seaborn as sns
import matplotlib as mpl
mpl.rcParams['pdf.fonttype'] = 42 # Output editable PDF text
### else
#import cloudpickle as pickle
import warnings
warnings.filterwarnings("ignore") # Error alert
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' #last_expr

In [3]:
from sklearn import model_selection
from sklearn.preprocessing import StandardScaler, Normalizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score, balanced_accuracy_score, recall_score, precision_score, roc_auc_score, average_precision_score

import math
import torch
from torch import nn
import torch.nn.functional as F
import torch.utils.data as TorchData
from torchmetrics import Accuracy
from pytorch_lightning import LightningModule, Trainer, seed_everything
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

from imblearn.over_sampling import RandomOverSampler, SMOTE

import warnings
warnings.filterwarnings('ignore')

SEED = 42
seed_everything(SEED, workers=True)

print("Using torch", torch.__version__)

Global seed set to 42


42

Using torch 1.10.1


In [4]:
microRNA = pd.read_csv('20220610_Results/1. diff_microRNA(FC2_FDR0.05).csv', index_col=0)['ID']

In [5]:
import cloudpickle as pickle
[mir2gene, GENE_LIST, GENE_MASK, gene2module, MODULE_LIST, MODULE_MASK,
 module2pathway, PATHWAY_LIST, PATHWAY_MASK] = pickle.load(open('20220610_Results/6. KEGG_MappingData.pkl', 'rb'))

In [6]:
len(PATHWAY_LIST), PATHWAY_LIST[:4]

(70, ['map00310', 'map00790', 'map00561', 'map01212'])

In [7]:
class MaskedModel(LightningModule):
    def __init__(self, mirN, geneN, gene_mask, moduleN, module_mask, pathwayN, pathway_mask, learning_rate):
        super(MaskedModel, self).__init__()
        self.save_hyperparameters()
        self.learning_rate = learning_rate
        # microRNA 2 Gene
        self.mir2gene = nn.Linear(mirN, geneN)
        self.mir2gene.weight.data = self.mir2gene.weight * gene_mask
        self.gene_mask = gene_mask
        self.gene_active = nn.ReLU() # ReLU Tanh
        self.gene_bn = nn.BatchNorm1d(geneN, eps=1e-05, momentum=0.5, affine=True)
        self.gene_dropout = nn.Dropout(p=0.8)
        
        ## Gene 2 Module
        self.gene2module = nn.Linear(geneN, moduleN)
        self.gene2module.weight.data = self.gene2module.weight * module_mask
        self.module_mask = module_mask
        self.module_active = nn.ReLU()
        self.module_bn = nn.BatchNorm1d(moduleN, eps=1e-05, momentum=0.5, affine=True)
        self.module_dropout = nn.Dropout(p=0.5)
        
        ## Module 2 Pathway
        self.module2pathway = nn.Linear(moduleN, pathwayN)
        self.module2pathway.weight.data = self.module2pathway.weight * pathway_mask
        self.pathway_mask = pathway_mask
        self.pathway_active = nn.ReLU()
        self.pathway_bn = nn.BatchNorm1d(pathwayN, eps=1e-05, momentum=0.5, affine=True)
        self.pathway_dropout = nn.Dropout(p=0.2)
        
        ## Output
        self.pathway2out = nn.Linear(pathwayN, 2)
        self.out_bn = nn.BatchNorm1d(2, eps=1e-05, momentum=0.5, affine=True)
        #self.sigmoid = nn.Sigmoid()
        
        # Result save
        self.gene_result = None
        self.module_result = None
        self.pathway_result = None
    
    def forward(self, x):
        ## microRNA 2 Gene
        #print(self.mir2gene.weight.data.shape, self.mir2gene.weight.shape, self.gene_mask.shape)
        self.mir2gene.weight.data = self.mir2gene.weight * self.gene_mask
        #print(self.mir2gene.weight.data.shape)
        gene_x = self.mir2gene(x)
        gene_x = self.gene_active(gene_x)
        gene_x = self.gene_bn(gene_x)
        gene_x = self.gene_dropout(gene_x)
        self.gene_result = gene_x # save result
        
        ## Gene 2 Module
        self.gene2module.weight.data = self.gene2module.weight * self.module_mask
        module_x = self.gene2module(gene_x)
        module_x = self.module_active(module_x)
        module_x = self.module_bn(module_x)
        module_x = self.module_dropout(module_x)
        self.module_result = module_x # save result
        
        ## Module 2 Pathway
        self.module2pathway.weight.data = self.module2pathway.weight * self.pathway_mask
        pathway_x = self.module2pathway(module_x)
        pathway_x = self.pathway_active(pathway_x)
        pathway_x = self.pathway_bn(pathway_x)
        pathway_x = self.pathway_dropout(pathway_x)
        self.pathway_result = pathway_x # save result
        
        ## Output
        out_x = self.pathway2out(pathway_x)
        out_x = self.out_bn(out_x)
        #out_x = self.sigmoid(out_x)
        return out_x
    
    def training_step(self, batch, batch_nb):
        x, y = batch
        loss = F.cross_entropy(self.forward(x), y)
        self.log("train_loss", loss, prog_bar=True)
        return loss
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=self.learning_rate)
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        logits = F.softmax(self(x))
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        probs = logits[:, 1]
        auc = roc_auc_score(y, probs)
        bac = balanced_accuracy_score(y, preds)
        aps = average_precision_score(y, probs)
        recall = recall_score(y, preds)
        precision = precision_score(y, preds)
        #self.auc(probs, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_bac", bac, prog_bar=True)
        self.log("val_auc", auc, prog_bar=True)
        self.log("val_aps", aps, prog_bar=True)
        self.log("val_rec", recall, prog_bar=True)
        self.log("val_pre", precision, prog_bar=True)
        return loss
    
    def validation_epoch_end(self, validation_step_outputs):
        avg_loss = torch.stack(validation_step_outputs).mean()
        #avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean() # Outputs are the values of return for validation
        self.logger.experiment.add_scalar('loss', avg_loss, self.current_epoch)
    
    def test_step(self, batch, batch_idx):
        # Here we just reuse the validation_step for testing
        return self.validation_step(batch, batch_idx)
    
    def predict_step(self, batch, batch_idx):
        x, y = batch
        logits = F.softmax(self(x))
        loss = F.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)
        probs = logits[:, 1]
        return preds, probs

In [8]:
model = MaskedModel.load_from_checkpoint(checkpoint_path="Temp/BioNet_SMOTE_val_loss.ckpt")

In [9]:
for name, param in model.named_parameters():
    print(f"Parameter {name}, shape {param.shape}")

Parameter mir2gene.weight, shape torch.Size([11418, 710])
Parameter mir2gene.bias, shape torch.Size([11418])
Parameter gene_bn.weight, shape torch.Size([11418])
Parameter gene_bn.bias, shape torch.Size([11418])
Parameter gene2module.weight, shape torch.Size([116, 11418])
Parameter gene2module.bias, shape torch.Size([116])
Parameter module_bn.weight, shape torch.Size([116])
Parameter module_bn.bias, shape torch.Size([116])
Parameter module2pathway.weight, shape torch.Size([70, 116])
Parameter module2pathway.bias, shape torch.Size([70])
Parameter pathway_bn.weight, shape torch.Size([70])
Parameter pathway_bn.bias, shape torch.Size([70])
Parameter pathway2out.weight, shape torch.Size([2, 70])
Parameter pathway2out.bias, shape torch.Size([2])
Parameter out_bn.weight, shape torch.Size([2])
Parameter out_bn.bias, shape torch.Size([2])


In [10]:
mir2gene_w = model.mir2gene.weight.data
gene2module_w = model.gene2module.weight.data
module2pathway_w = model.module2pathway.weight.data
pathway2out_w = model.pathway2out.weight.data

In [11]:
### Select

In [12]:
X_train_ = pd.read_csv('Breast cancer/E-GEOD-73002.csv', index_col=0)
X_train_torch = torch.from_numpy(X_train_[microRNA].values).type(torch.FloatTensor)

In [13]:
X_test = pd.read_csv('20220610_Results/1. X_test.csv', index_col=0)
y_test = pd.read_csv('20220610_Results/1. y_test.csv', index_col=0)
X_train_ = X_test.copy()
X_train_['Label'] = y_test['Label']
X_train_.head(2)
X_train_torch = torch.from_numpy(X_test[microRNA].values).type(torch.FloatTensor)

,hsa-let-7a-2-3p,hsa-let-7a-3p,hsa-let-7a-5p,hsa-let-7b-3p,hsa-let-7b-5p,hsa-let-7c-3p,hsa-let-7c-5p,hsa-let-7d-3p,hsa-let-7d-5p,hsa-let-7e-3p,...,hsa-miR-6813-5p,hsa-miR-8071,hsa-miR-6893-5p,hsa-miR-642a-3p,hsa-miR-1199-5p,hsa-miR-3162-5p,hsa-miR-6805-3p,hsa-miR-4467,hsa-miR-6076,Label
GSM1878567,2.721157,0.581391,0.581391,3.284674,0.581391,2.350276,0.581391,2.356790,0.581391,0.581391,...,5.958955,7.592305,8.284014,7.705975,4.332095,6.766729,6.98926,10.266124,7.558272,0
GSM1877697,1.790763,0.898406,0.898406,2.508507,0.898406,1.153854,0.898406,2.249823,0.898406,0.898406,...,5.758528,8.276442,7.676088,7.763261,4.790422,7.435764,6.57151,10.160213,8.755441,0


In [14]:
gene_x = model.mir2gene(X_train_torch)
gene_x = model.gene_active(gene_x)
gene_x = model.gene_bn(gene_x)
gene_exp = gene_x

module_x = model.gene2module(gene_x)
module_x = model.module_active(module_x)
module_x = model.module_bn(module_x)
module_exp = module_x
        
pathway_x = model.module2pathway(module_x)
pathway_x = model.pathway_active(pathway_x)
pathway_x = model.pathway_bn(pathway_x)
pathway_exp = pathway_x

In [15]:
gene_exp_df = pd.DataFrame(np.array(gene_exp.detach().numpy()), index=X_train_.index, columns=GENE_LIST)
module_exp_df = pd.DataFrame(np.array(module_exp.detach().numpy()), index=X_train_.index, columns=MODULE_LIST)
pathway_exp_df = pd.DataFrame(np.array(pathway_exp.detach().numpy()), index=X_train_.index, columns=PATHWAY_LIST)

In [16]:
import cloudpickle as pickle
pickle.dump([X_train_, gene_exp_df, module_exp_df, pathway_exp_df, mir2gene_w, gene2module_w,
             module2pathway_w, pathway2out_w], open('20220610_Results/8. Exp_and_Weight_df_test.pkl', 'wb'))

In [17]:
pathway_exp_df.head(2)

,map00310,map00790,map00561,map01212,map00480,map05016,map00563,map00601,map00564,map00140,...,map00190,map00020,map00061,map00130,map00532,map00280,map00680,map00515,map00350,map00330
GSM1878567,0.000309,-0.385335,-1.375007,2.678093,0.060196,0.224582,-1.189325,0.350027,0.750644,-1.075417,...,-2.218088,-1.618479,0.227646,-0.15073,1.067602,2.402193,-0.376911,0.07354,1.383809,-0.716437
GSM1877697,0.000309,-0.385335,-0.916085,1.908162,0.060196,0.224582,-1.189325,0.350027,0.568939,-1.075417,...,-2.497192,-1.618479,0.227646,-0.15073,1.055862,-0.238823,-0.220332,0.07354,1.217697,-0.716437


In [18]:
## Output
out_x = model.pathway2out(pathway_x)
out_x = model.out_bn(out_x)

In [20]:
logits = F.softmax(out_x)
preds = torch.argmax(logits, dim=1)
probs = logits[:, 1]

In [23]:
preds[:10]

tensor([0, 0, 0, 1, 0, 0, 0, 0, 0, 0])

In [22]:
probs

tensor([0.0044, 0.0067, 0.0125,  ..., 0.0432, 0.1396, 0.9974],
       grad_fn=<SelectBackward0>)